In [4]:
import pandas as pd
import os

## Homework from Week2

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and 
model management.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 2.13.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [9]:
raw_data_path = "data/raw/green"
dest_path = "output"
script_path = "scripts"

In [10]:
df_01_2023_green = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet"
)
df_02_2023_green = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet"
)
df_03_2023_green = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"
)

if not os.path.exists(raw_data_path):
    # If the directory does not exist, create it
    os.makedirs(raw_data_path)
    print(f"Directory '{raw_data_path}' created.")
else:
    print(f"Directory '{raw_data_path}' already exists.")


df_01_2023_green.to_parquet(f"{raw_data_path}/green_tripdata_2023-01.parquet")
df_02_2023_green.to_parquet(f"{raw_data_path}/green_tripdata_2023-02.parquet")
df_03_2023_green.to_parquet(f"{raw_data_path}/green_tripdata_2023-03.parquet")

Directory 'data/raw/green' created.


In [14]:
!python {script_path}/preprocess_data.py --raw_data_path {raw_data_path} --dest_path {dest_path}

In [15]:
file_count = len([name for name in os.listdir(dest_path) if os.path.isfile(os.path.join(dest_path, name))])
print(f"Number of files in '{dest_path}': {file_count}")

Number of files in 'output': 4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10


In [19]:
!python {script_path}train.py --data_path {dest_path}

2024/05/29 17:49:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/05/29 17:49:17 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [20]:
!mlflow ui --backend-store-uri sqlite:///mlruns.db

[2024-05-29 17:53:26 +0300] [7309] [INFO] Starting gunicorn 22.0.0
[2024-05-29 17:53:26 +0300] [7309] [INFO] Listening at: http://127.0.0.1:5000 (7309)
[2024-05-29 17:53:26 +0300] [7309] [INFO] Using worker: sync
[2024-05-29 17:53:26 +0300] [7310] [INFO] Booting worker with pid: 7310
[2024-05-29 17:53:26 +0300] [7311] [INFO] Booting worker with pid: 7311
[2024-05-29 17:53:26 +0300] [7312] [INFO] Booting worker with pid: 7312
[2024-05-29 17:53:26 +0300] [7313] [INFO] Booting worker with pid: 7313
^C
[2024-05-29 17:55:16 +0300] [7309] [INFO] Handling signal: int
[2024-05-29 17:55:16 +0300] [7312] [INFO] Worker exiting (pid: 7312)
[2024-05-29 17:55:16 +0300] [7311] [INFO] Worker exiting (pid: 7311)
[2024-05-29 17:55:16 +0300] [7310] [INFO] Worker exiting (pid: 7310)
[2024-05-29 17:55:16 +0300] [7313] [INFO] Worker exiting (pid: 7313)


![min_samples_split](https://d3dehtdmp2rwcw.cloudfront.net/ms_562632/apO5Jdx7eNOa9zu7hNGSMfhlrtkbXO/Monosnap%2BMLflow%2B2024-05-29%2B17-56-08.png?Expires=1716994800&Signature=CkAfYAjcr3wBqCOafoC7~YyXRDmIiEHWyYsmOLLahnOm-K2Jx5GQakrGEJWOdxznswYdHzj7JsRHkmQ1CjvdxYBw0meIMKNUVuqCnnZjmAju3oiG7RznU5-RDnN5K2PfvwjSkxZkOOaNQh-49YyyLlkVsJ7Ue9JS52T-lOR5uIQN1jWUZXoE06CLZLj-TZnRz6H~~OUw19kaE9CgBBVNy8j3fg6rd2aOmRmaB0jzCahLuL31mgOGkhc74Y~FPmFNcHubeB3QsPMSw~rWAA9meOHa26-yImKaOAt9YfTThIZpkxlCgQSBND-WyiO4H8dvrXSr71HnknjIdCAO0zFGug__&Key-Pair-Id=APKAJBCGYQYURKHBGCOA)

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


In [21]:
!mlflow server --backend-store-uri sqlite:///mlruns.db --default-artifact-root ./artifacts

[2024-05-29 18:01:58 +0300] [7484] [INFO] Starting gunicorn 22.0.0
[2024-05-29 18:01:58 +0300] [7484] [INFO] Listening at: http://127.0.0.1:5000 (7484)
[2024-05-29 18:01:58 +0300] [7484] [INFO] Using worker: sync
[2024-05-29 18:01:58 +0300] [7485] [INFO] Booting worker with pid: 7485
[2024-05-29 18:01:58 +0300] [7486] [INFO] Booting worker with pid: 7486
[2024-05-29 18:01:58 +0300] [7487] [INFO] Booting worker with pid: 7487
[2024-05-29 18:01:59 +0300] [7488] [INFO] Booting worker with pid: 7488
^C
[2024-05-29 18:02:11 +0300] [7484] [INFO] Handling signal: int
[2024-05-29 18:02:11 +0300] [7485] [INFO] Worker exiting (pid: 7485)
[2024-05-29 18:02:11 +0300] [7487] [INFO] Worker exiting (pid: 7487)
[2024-05-29 18:02:11 +0300] [7486] [INFO] Worker exiting (pid: 7486)
[2024-05-29 18:02:11 +0300] [7488] [INFO] Worker exiting (pid: 7488)


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336
